In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Employees')

+----+----------+
| id |   name   |
+----+----------+
| 1  |  Alice   |
| 7  |   Bob    |
| 11 |   Meir   |
| 90 | Winston  |
| 3  | Jonathan |
+----+----------+


In [3]:
display_table('EmployeeUNI')

+----+-----------+
| id | unique_id |
+----+-----------+
| 3  |     1     |
| 11 |     2     |
| 90 |     3     |
+----+-----------+


### Write an SQL query to show the unique ID of each user, If a user does not have a unique ID replace just show `null`.

```
+-----------+----------+
| unique_id | name     |
+-----------+----------+
| null      | Alice    |
| null      | Bob      |
| 2         | Meir     |
| 3         | Winston  |
| 1         | Jonathan |
+-----------+----------+
```

In [4]:
%%sql 

SELECT unique_id, name
FROM Employees e 
LEFT JOIN EmployeeUNI eu ON eu.id = e.id

unique_id,name
None,Alice
None,Bob
2,Meir
3,Winston
1,Jonathan


# Using Pandas

In [5]:
employee_query = %sql SELECT * FROM Employees # type: ignore
id_query = %sql SELECT * FROM EmployeeUNI  # type: ignore
employee_df = employee_query.DataFrame()
id_df = id_query.DataFrame()
display(employee_df, id_df)

,id,name
0,1,Alice
1,7,Bob
2,11,Meir
3,90,Winston
4,3,Jonathan


,id,unique_id
0,3,1
1,11,2
2,90,3


## merge()

In [6]:
# merge without optional args
pd.merge(employee_df, id_df) 

,id,name,unique_id
0,11,Meir,2
1,90,Winston,3
2,3,Jonathan,1


In [7]:
pd.merge(employee_df, id_df, on='id', how='left')

,id,name,unique_id
0,1,Alice,NaN
1,7,Bob,NaN
2,11,Meir,2.0
3,90,Winston,3.0
4,3,Jonathan,1.0


In [8]:
# move the id_df to change the order of columns to match the sql output
pd.merge(id_df, employee_df, on='id', how='right')

,id,unique_id,name
0,1,NaN,Alice
1,7,NaN,Bob
2,11,2.0,Meir
3,90,3.0,Winston
4,3,1.0,Jonathan


In [9]:
# id is optional
pd.merge(id_df, employee_df, how='right')

,id,unique_id,name
0,1,NaN,Alice
1,7,NaN,Bob
2,11,2.0,Meir
3,90,3.0,Winston
4,3,1.0,Jonathan


In [17]:
# match the sql output
pd.merge(id_df, employee_df, how='right')[['unique_id', 'name']] \
    .set_index('unique_id', drop=True)

,name
unique_id,
NaN,Alice
NaN,Bob
2.0,Meir
3.0,Winston
1.0,Jonathan


## join()

In [28]:
id_df.set_index('id').join(employee_df.set_index('id'), how='right') \
    .set_index('unique_id', drop=True)

,name
unique_id,
NaN,Alice
NaN,Bob
2.0,Meir
3.0,Winston
1.0,Jonathan
